In [42]:
import os
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from helpers.get_llm import LLM
from tools.tools import tools_list
from helpers.load_prompt import load_prompt_from_yaml
from tools.state_tool import get_user_state_
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage , AnyMessage
from langgraph.prebuilt import create_react_agent
from typing_extensions import TypedDict
import random
from typing import Literal , Annotated
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tool_node , tools_condition , tool_validator , ToolNode

In [2]:

llm = LLM.get_groq_llm()



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


In [3]:


class State(TypedDict):
    graph_state: str
    
    
def node_1(state):
    print("---Node 1---")
    return {"graph_state": state['graph_state'] +" I am"}

def node_2(state):
    print("---Node 2---")
    return {"graph_state": state['graph_state'] +" happy!"}

def node_3(state):
    print("---Node 3---")
    return {"graph_state": state['graph_state'] +" sad!"}





In [4]:


def decide_mood(state) -> Literal["node_2", "node_3"]:
    user_input = state['graph_state'] 
    if random.random() < 0.5:
        return "node_2"
    return "node_3"





In [5]:

builder = StateGraph(State)


# Adding Nodes
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)


# Logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1" , decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)



In [6]:

#Add
graph = builder.compile()


# display(Image(graph.get_graph().draw_mermaid_png()))

op = graph.invoke({"graph_state": "Hi Im Chinmay"})


print("Final State:",op)

---Node 1---
---Node 2---
Final State: {'graph_state': 'Hi Im Chinmay I am happy!'}


In [7]:
from langchain_core.messages import HumanMessage, SystemMessage , AIMessage

messages = [AIMessage(content="Hi", name="Model")]
messages.extend([HumanMessage(content="I am", name="User")])
messages.extend([AIMessage(content="happy!", name="Model")])
messages.extend([AIMessage(content="How can i Help You", name="Model")])
messages.extend([HumanMessage(content="I want to know some craxy facts", name="User")])

In [8]:
print(messages)
res = llm.invoke(messages)
res

[AIMessage(content='Hi', additional_kwargs={}, response_metadata={}, name='Model'), HumanMessage(content='I am', additional_kwargs={}, response_metadata={}, name='User'), AIMessage(content='happy!', additional_kwargs={}, response_metadata={}, name='Model'), AIMessage(content='How can i Help You', additional_kwargs={}, response_metadata={}, name='Model'), HumanMessage(content='I want to know some craxy facts', additional_kwargs={}, response_metadata={}, name='User')]


AIMessage(content='\n<think>\nOkay, the user wants to know some crazy facts. Let me think of some interesting ones.\n\nFirst, maybe the one about the giraffe\'s neck and the giraffe\'s tongue. The tongue can be up to 18 inches long, which is really long. That\'s a cool fact.\n\nThen, bananas have their own DNA? Wait, actually, bananas as plants have a different chromosome count. Maybe that\'s a better way to phrase it. They have three copies of each chromosome, making them triploid. That might be a bit technical, but still interesting.\n\nOh, right! Venus flytraps can\'t survive on their own without carnivory. They need the insects to get nitrogen. That\'s a good one.\n\nThe octopus with the detachable arm. Yeah, some species can drop an arm to escape predators. The arm even keeps moving. That\'s wild.\n\nShrimp and their "memory" in the ocean. The chemical they release when stressed affects other shrimp. Maybe I should explain that more clearly.\n\nThe black swan theory from Nassim Ta

In [11]:
res.content.split("</think>")[-1].replace("\n\n","")

'Here are some *crazy* facts to blow your mind 🤯:1. **Giraffes have blue tongues** – Their tongues can be up to **18 inches (45 cm) long** and are dark blue-black, which scientists think helps protect them from sunburn while eating.  2. **Bananas are almost like cloning** – Commercial bananas are sterile (they don’t have seeds) and are essentially clones of each other, making them vulnerable to disease. Plus, wild bananas are **full of giant, hard seeds** and taste terrible.  3. **Venus flytraps can’t survive on sunlight alone** – They’re carnivorous because they grow in nutrient-poor soil, so they need to eat insects for nitrogen.  4. **Some octopuses can *drop an arm* like a lizard’s tail** – The **blue-ringed octopus** can detach an arm to escape predators, and the arm *continues to wriggle* to distract the predator.  5. **Shrimp can "remember" each other** – When stressed, shrimp release a chemical that "warns" other shrimp of danger, and they react even if they’ve never met the st

In [14]:
llm_with_tools = llm.bind_tools(tools_list)

In [17]:
Tool_call = llm_with_tools.invoke([HumanMessage(content="Whats Happeing In germnay rn", name="User")])

In [21]:
Tool_call.additional_kwargs['tool_calls']

[{'id': 'call_f80x',
  'function': {'arguments': '{"query": "current events and news in Germany today"}',
   'name': 'search'},
  'type': 'function'}]

In [25]:
class State(TypedDict):
    messages : Annotated[list[AnyMessage], add_messages]

# TOOL CALL GRAPH

In [ ]:
class State(MessagesState):
    pass

In [38]:
def tool_call_llm(state : MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
    # return state

In [ ]:
builder = StateGraph(State)
builder.add_node("tool_call_llm", tool_call_llm)
builder.add_edge(START, "tool_call_llm")
builder.add_edge("tool_call_llm", END)
graph = builder.compile()

In [29]:
message = graph.invoke({"messages": HumanMessage(content="Hello", name="User")})
message

{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, name='User', id='0e2b79c2-0b55-41ad-94df-1a13df1c099b'),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 445, 'total_tokens': 522, 'completion_time': 0.191892794, 'prompt_time': 0.024129991, 'queue_time': 0.10252348700000001, 'total_time': 0.216022785}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_27d5db8d87', 'finish_reason': 'stop', 'logprobs': None}, id='run-4cfd5b0d-31da-4b4b-ad44-ee243074dd9f-0', usage_metadata={'input_tokens': 445, 'output_tokens': 77, 'total_tokens': 522})]}

In [30]:
message = graph.invoke({"messages": HumanMessage(content="whats happening In Germany Rn", name="User")})
message

{'messages': [HumanMessage(content='whats happening In Germany Rn', additional_kwargs={}, response_metadata={}, name='User', id='7ae77699-d54d-437b-ae00-998d157af777'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9ht1', 'function': {'arguments': '{"query": "what\'s happening in Germany right now"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 223, 'prompt_tokens': 451, 'total_tokens': 674, 'completion_time': 0.540232196, 'prompt_time': 0.029495052, 'queue_time': 0.10246565700000002, 'total_time': 0.569727248}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_961d4d54da', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4fb79704-59dc-429a-a132-00c4dbd561c0-0', tool_calls=[{'name': 'search', 'args': {'query': "what's happening in Germany right now"}, 'id': 'call_9ht1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 451, 'output_tokens': 223, 'total_tokens': 674})]}

In [32]:
State["messages"]

__main__.State['messages']

# Routers

In [45]:
builder = StateGraph(State)
builder.add_node("tool_call_llm", tool_call_llm)
builder.add_edge("tools" , ToolNode(tools_list))
builder.add_edge(START, "tool_call_llm")
builder.add_conditional_edges
builder.add_edge("tools", END)
graph = builder.compile()

ValueError: Found edge starting at unknown node 'tools'